**Turbines for ebb-flow hydropower; identifying the optimum start-time after high tide for maximum safe power generation, and identifying the required build depth and number.**


SORTEDDDDD!!!!!! YYEEEESSSSSS!!!!!

Evaluating the variation in: maximum energy output; average energy output; depth of turbine needed to prevent cavitation; and optimal start time for best power generation; with the number of turbines.

Case considering an ebb-flow generation hydropower scheme on the Severn estuary.
Numbers below for the Middle Barrage.

In [ ]:
max_elev_caisson_base_max_energy_list = []
energy_total_max_list = []
start_time_max_energy_list = []
No_Turbines_LIST = []

Rm = 8
D = 8.5
A = 480*10**6

import numpy as np
import matplotlib.pyplot as plt
from sympy import S,symbols, printing

def specdischarge_curve_BC(n):
  return - (3.5449 * 10**(-8) * n**(3)) + (2.4725228 * 10**(-5) * n**(2)) - 0.006622404081*n + 5.15217197748
def specdischarge_curve_AB(n):
  return 0.017902021773*n + 0.318184551581

def efficiency_curve_AB(n):
  return - (1.0792 * 10**(-8) * n**(5)) + (1.0260084 * 10**(-5) * n**(4)) - (0.003872045356 * n**(3)) + (0.722541346803 * n**(2)) - 66.62253930374*n + 2527.667771731353
def efficiency_curve_BC(n):
  return (5.1 * 10**(-11) * n**(5)) - (1.08811 * 10**(-7) * n**(4)) + (8.7962193 * 10**(-5) * n**(3)) - (0.034381133561 * n**(2)) + 6.406373184409*n - 373.328017605249

def cavlim_curve_hardcode(x):
  return 0.379978349153 * x**2 - 0.862889414038 * x + 1.415823731564

LunarPeriod = 12.42 #in hours

#TidalRangeMoon = 8

#Rm = 8 #TidalRangeMoon in m

roh_fw = 999 #density fresh water in kg/m^3
roh_sw = 1030 #density salt water in kg/m^3
g = 9.81 #gravitational acc in ms^-2
press_atm = 100 #atmospheric pressure in kPa

#No_Turbines = 400 #number of turbines
f = 50 #frequency turbine electricity output in Hz
eta_generator = 0.95 #efficiency of generator, assumed constant

#A = 1000 * 10**6 #m^2 surface area of whole basin
print("Surface area of basin, A = {} [m^2]".format(A))
#D = 8 # in m ???

n_opt = (Rm/8)**0.5 * (7.5/D) * 60
print("Optimal turbine speed, n_opt = {} [rpm]".format(n_opt))
n_opt = int(np.floor(n_opt)) #note np.floor and np.ceil seem to have the opposite effects here than in the documentation ??????
print("Optimal turbine speed, rounded, n_opt = {} [rpm]".format(n_opt))
Np_trial = 2*f*60/n_opt
print(Np_trial)
Np_trial = int(np.floor(Np_trial/2)*2)
print("Trial number of turbine poles, accounting for paired nature, Np_trial = {}".format(Np_trial))
n_revised = 2*f*60/Np_trial
print("Revised turbine speed, accounting for turbine pole number, n_revised = {}".format(n_revised))
print("")





#Varying number of turbines
No_Turbines_max = 400
No_Turbines_min = 10
delta_Turbines = 10

#No_Turbine_list = list(range(No_Turbines_min, No_Turbines_max + 1, delta_Turbines))
#No_Turbine_list = np.linspace(delta_Turbines,No_Turbines_max,int((No_Turbines_max-No_Turbines_min)/(delta_Turbines) + 1))
#print(No_Turbine_list)
#print("\n")

#for No_Turbines in No_Turbine_list:
#  exxxx
for No_Turbines in range(No_Turbines_min, No_Turbines_max + 1, delta_Turbines):
  No_Turbines_LIST.append(No_Turbines)

  #do iteration over set of t, revise how to do this efficiently
  start_time_list = []
  energy_total_list = []
  max_elev_caisson_base_list = []
  count_num = 0

  #tstart = 12.42*(1/64)
  #tend = (63*12.42/64)
  t_increment = LunarPeriod*(1/160)
  tstart = LunarPeriod*(1/64)
  tend = LunarPeriod*(63/64)

  while tstart<tend:
    #count_num+=1
    #print(count_num)
    start_time_list.append(tstart)
    #------------------------

    t_list = []
    hb_list = []
    hs_list = []
    H_list = []
    Q_list = []
    P_N_list = []
    E_N_list = []
    elev_caisson_base_list = []
    #print(((12.42-t0)/(12.42/64)))
    t_list = np.linspace(tstart,tend,int((tend-tstart)/(t_increment) + 1))
    #print(t_list)
    #print(type(t_list))



    for t in t_list:
      state_ON = True
      if t==tstart:
        t0 = tstart
      else:
        t0 = t0
      #print("Time, interval start, t = {} [h]".format(t))
      delta_t = t-t0

      #np.cos takes input in radians
      hs = (Rm / 2) * np.cos((2*np.pi*t)/12.42)
      #print("Lunar tide height, hs = {} [m]".format(hs))

      #Find hb
      hb_max = Rm/2
      #use an exception or better check for this
      while True:
        if t==t0:
          hb0 = hb_max
          break
        else:
          break

      #print("Retained water level, hb0 = {} [m]".format(hb0))
      #Need to catch this properly.....

      H0 = hb0 - hs
      #print("Head_0 at start of interval, H0 = {} [m]".format(H0))

      hb_list.append(hb0)
      hs_list.append(hs)
      H_list.append(H0)

      #Checking if head difference is positive
      while True:
        if hb0 > hs:
          state_ON = True
          break
        else:
          state_ON = False
          break


      unit_speed = (n_revised * D)/(H0**0.5)
      #print("Unit speed through turbine, unit_speed = {} [??]".format(unit_speed)) # UNITS!!!!!!

      #Catching unsuitable numbers
      while True:
        if unit_speed > 234:
          if unit_speed <=514:
            sd_val = specdischarge_curve_BC(unit_speed)
            eff_val = efficiency_curve_BC(unit_speed)
            break
          else:
            print("Input unit speed too high")
            sd_val = np.nan
            eff_val = np.nan
            state_ON = False
            break
        else:
          if unit_speed < 140:
            print("Input unit speed too low")
            sd_val = np.nan
            eff_val = np.nan
            state_ON = False
            break
          else:
            sd_val = specdischarge_curve_AB(unit_speed)
            eff_val = efficiency_curve_AB(unit_speed)
            break

      #Catching unsuitable numbers, suspending operation
      while True:
        if state_ON == False:
          Q=0
          elev_caissonbase = np.nan
          break
        else:
      #Checking runner blade root submergence
          sigma_cav_lim = cavlim_curve_hardcode(sd_val)
          d_rootsub = sigma_cav_lim*H0 - (press_atm * 10**3)/(roh_sw*g) #depth submerged to runner blade root [m]
          elev_root = (hb0 - d_rootsub)
          #Cavitation check:
          D_runnerhub = D/3 #Diameter of turbine runner hub
          d_turbineaxis = (d_rootsub + 0.5*D_runnerhub)
          elev_turbineaxis = (hb0 - d_turbineaxis)
          elev_caissonbase = (hb0 - d_turbineaxis - 1.4*D)
          #print("Normalised max elevation of turbine axis to prevent cavitation, for the head difference at the start of the interval, elev_turbineaxis = {} [m]".format(elev_turbineaxis))
          print("Normalised max elevation of caisson base to prevent cavitation, elev_caissonbase = {} [m], for head difference at interval start, and caisson base at 1.4D below turbine axis".format(elev_caissonbase))
          Q = sd_val * (D**2 * H0**0.5) #m^3/s
          break

      elev_caisson_base_list.append(elev_caissonbase)
      #print("Volumetric flow through turbine, Q = {} [m^3 / s ???]".format(Q))
      Q_list.append(Q)

      delta_hb = (-No_Turbines*(Q)*(delta_t * 60**2))/A  # matching units
      #print("Change in upstream head over interval, delta_hb = {} [m]".format(delta_hb))
      hb1 = hb0 + delta_hb
      #print("Upstream head, at end of interval, hb1 = {} [m] \n".format(hb1))
      #Total power output from all generators:
      Pow_Out_N = No_Turbines*roh_fw*g*Q*H0*eff_val*eta_generator
      P_N_list.append(Pow_Out_N)
      #print("Power output total, for all turbines, Pow_Out_N = {} [W = kgm^2/s^3]".format(Pow_Out_N))
      #print("Power output total, for all turbines, Pow_Out_N = {} [MW = 10^6*kgm^2/s^3]".format(Pow_Out_N/(10**6)))
      delta_E_N = Pow_Out_N*(delta_t * 60**2) #Be careful about hour to seconds conversion
      E_N_list.append(delta_E_N)
      #print("Energy output total, for all turbines over the interval, delta_E_N = {} [J = kgm^2/s^2]".format(delta_E_N))
      #print("Energy output total, for all turbines over the interval, delta_E_N = {} [MJ = 10^6*kgm^2/s^2]".format(delta_E_N/(10**6)))
      #print("------------------------------------------------------------")
      hb0 = hb1
      t0 = t

    print(elev_caisson_base_list)
    max_elev_caisson_base = np.nanmin(elev_caisson_base_list)
    max_elev_caisson_base_list.append(max_elev_caisson_base)

    Energy_total = 0
    for i in E_N_list:
      #Energy_total += i
      Energy_total += np.nan_to_num(i)

    #print("Total energy output over operation, E_total = {} [J]\nThis is the area under the power graph\n".format(Energy_total))
    #print("Total energy output over operation, E_total = {} [GJ]\nThis is the area under the power graph\n".format(Energy_total/(10**9)))
    #print("Be careful about the electrical losses at low power outputs reported by Baker on p.178 of Economic of Tidal Power")

    energy_total_list.append(Energy_total)
    tstart += (t_increment)


  print(energy_total_list)
  for i,j in (zip(start_time_list, energy_total_list)):
    print("For start time {} [h], end time {} [h], we have energy output {} [GJ]".format(i,tend,(j/(10**9))))

  energy_total_list_max = np.nanmax(energy_total_list)
  energy_total_list_max_index = energy_total_list.index(energy_total_list_max)
  start_time_max_energy = start_time_list[energy_total_list_max_index]
  max_elev_caisson_max_energy = max_elev_caisson_base_list[energy_total_list_max_index]

  #print("\nFor D = {} [m], No_T = {}, start time {} [h], end time {} [h], we have the maximum energy output {} [GJ] and max caisson elev {} [m]".format(D,No_Turbines,start_time_max_energy,tend,(energy_total_list_max/(10**9)),max_elev_caisson_max_energy))
  #print("\nWe also have maximum caisson elevation {} [m]".format(max_elev_caisson_max_energy))
  print("\nRm,D,No_Turbines,start_time_max_energy,tend,(energy_total_list_max/(10**9)),max_elev_caisson_max_energy")
  print("{},{},{},{},{},{},{}".format(Rm,D,No_Turbines,start_time_max_energy,tend,(energy_total_list_max/(10**9)),max_elev_caisson_max_energy))

  print(max_elev_caisson_base_list)
  energy_total_max_list.append(energy_total_list_max)
  max_elev_caisson_base_max_energy_list.append(max_elev_caisson_max_energy)
  start_time_max_energy_list.append(start_time_max_energy)

  #Plot
  plt.figure()
  plt.plot(start_time_list, max_elev_caisson_base_list, 'g-', linewidth = 1, label = "hs")
  #plt.plot(t_list, hb_list, color = 'r', marker = 'x', linestyle = 'dashed', label = "hb")
  plt.xlabel("start time [h]")
  plt.ylabel("maximum elevation caisson [m]")
  plt.legend()
  plt.grid(True)
  plt.title("max allowable caisson elevation vs start time [m]")
  plt.show()

    #plt.figure()
    #plt.plot(t_list, P_N_list, 'g-', linewidth = 1, label = "Power output")
    #plt.xlabel("time [h]")
    #plt.ylabel("power output [W]")
    #plt.legend()
    #plt.grid(True)
    #plt.title("y vs E [m]")
    #plt.show()

tstart = 12.42*(1/64)
tend = (63*12.42/64)

In [ ]:
print(max_elev_caisson_base_max_energy_list)
print(energy_total_max_list)
print(start_time_max_energy_list)
print(No_Turbines_LIST)

avgenergyturbine_max_list = list([nrg/notrb for nrg,notrb in zip(energy_total_max_list,No_Turbines_LIST)])
print(avgenergyturbine_max_list)

In [ ]:
#Plot
plt.figure()
plt.plot(No_Turbines_LIST, max_elev_caisson_base_max_energy_list, 'g-', linewidth = 1, label = "Max elev caisson base")
#plt.plot(t_list, hb_list, color = 'r', marker = 'x', linestyle = 'dashed', label = "hb")
plt.xlabel("no turbines []")
plt.ylabel("maximum elevation caisson [m]")
plt.legend()
plt.grid(True)
plt.title("max allowable caisson elevation vs no turbines")
plt.show()
#Plot
plt.figure()
plt.plot(No_Turbines_LIST, energy_total_max_list, 'g-', linewidth = 1, label = "Max energy")
#plt.plot(t_list, hb_list, color = 'r', marker = 'x', linestyle = 'dashed', label = "hb")
plt.xlabel("no turbines []")
plt.ylabel("maximum energy [?]")
plt.legend()
plt.grid(True)
plt.title("max energy vs no turbines")
plt.show()
#Plot
plt.figure()
plt.plot(No_Turbines_LIST, avgenergyturbine_max_list, 'g-', linewidth = 1, label = "Avg energy per turbine")
#plt.plot(t_list, hb_list, color = 'r', marker = 'x', linestyle = 'dashed', label = "hb")
plt.xlabel("no turbines []")
plt.ylabel("avg energy per turbine, max energy production [?]")
plt.legend()
plt.grid(True)
plt.title("avg energy per turbine vs no turbines")
plt.show()
#Plot
plt.figure()
plt.plot(No_Turbines_LIST, start_time_max_energy_list, 'g-', linewidth = 1, label = "Start time")
#plt.plot(t_list, hb_list, color = 'r', marker = 'x', linestyle = 'dashed', label = "hb")
plt.xlabel("no turbines []")
plt.ylabel("start time [h]")
plt.legend()
plt.grid(True)
plt.title("start time vs no turbines")
plt.show()

Looking at the details again for the